In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Часть 1. Предобработка
Предобработка такая же что и в первом дз

In [3]:
data = pd.read_csv('./Shanghai_HMT_2010.csv')
data.head()

,No,year,month,day,hour,season,PM_Jingan,PM_US Post,PM_Xuhui,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
0,1,2010,1,1,0,4,NaN,NaN,NaN,-6.0,59.48,1026.1,1.0,cv,1.0,0.0,0.0
1,2,2010,1,1,1,4,NaN,NaN,NaN,-6.0,59.48,1025.1,1.0,SE,2.0,0.0,0.0
2,3,2010,1,1,2,4,NaN,NaN,NaN,-7.0,59.21,1025.1,0.0,SE,4.0,0.0,0.0
3,4,2010,1,1,3,4,NaN,NaN,NaN,-6.0,63.94,1024.0,0.0,SE,5.0,0.0,0.0
4,5,2010,1,1,4,4,NaN,NaN,NaN,-6.0,63.94,1023.0,0.0,SE,8.0,0.0,0.0


In [4]:
null_value_stats = data.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

PM_Jingan        27884
PM_US Post       18545
PM_Xuhui         27395
DEWP                13
HUMI                13
PRES                28
TEMP                13
cbwd                12
Iws                 12
precipitation     4009
Iprec             4009
dtype: int64

Выбрасываем наны и колокну с номерами

In [5]:
work_data = data.drop(data[data["PRES"].isnull()].index)
work_data = work_data.drop(['No'], axis=1)
work_data = work_data.dropna()

In [6]:
date = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
work_data = work_data.join(pd.DataFrame({'date' : date} ) )
work_data['date'] = work_data['date'].astype('int').astype('float'); 
work_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,year,month,day,hour,season,PM_Jingan,PM_US Post,PM_Xuhui,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec,date
26304,2013,1,1,0,4,66.0,70.0,71.0,-5.0,69.00,1023.0,0.0,SW,60.0,0.0,0.0,1.356998e+18
26305,2013,1,1,1,4,67.0,76.0,72.0,-5.0,69.00,1023.0,0.0,SW,62.0,0.0,0.0,1.357002e+18
26307,2013,1,1,3,4,73.0,78.0,74.0,-4.0,74.41,1023.0,0.0,SW,65.0,0.0,0.0,1.357009e+18
26308,2013,1,1,4,4,75.0,77.0,77.0,-4.0,80.04,1023.0,-1.0,SW,68.0,0.0,0.0,1.357013e+18
26309,2013,1,1,5,4,73.0,78.0,80.0,-4.0,80.04,1023.0,-1.0,SW,70.0,0.0,0.0,1.357016e+18


In [7]:
print(work_data.drop(["PRES"], axis=1).dtypes)
cat_f_ind = np.where(work_data.drop(["PRES"], axis=1).dtypes != float)[0]

year               int64
month              int64
day                int64
hour               int64
season             int64
PM_Jingan        float64
PM_US Post       float64
PM_Xuhui         float64
DEWP             float64
HUMI             float64
TEMP             float64
cbwd              object
Iws              float64
precipitation    float64
Iprec            float64
date             float64
dtype: object


# Часть 2. Предсказание давления

In [8]:
from catboost import CatBoostRegressor
from catboost import Pool, cv
from sklearn.model_selection import train_test_split

target = 'PRES';
cols = work_data.columns

X_train, X_test, y_train, y_test = train_test_split(work_data[cols[cols!=target]], work_data[target], test_size=0.15)

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [13]:
model = CatBoostRegressor(max_depth=12, iterations=1200, task_type="GPU",loss_function='RMSE')
model.fit(X_train, y_train, cat_features=cat_f_ind, eval_set=(X_test, y_test))

Learning rate set to 0.080665
0:	learn: 8.3634131	test: 8.3476304	best: 8.3476304 (0)	total: 530ms	remaining: 10m 35s
1:	learn: 7.8110666	test: 7.7936957	best: 7.7936957 (1)	total: 926ms	remaining: 9m 14s
2:	learn: 7.3136647	test: 7.2916053	best: 7.2916053 (2)	total: 1.33s	remaining: 8m 50s
3:	learn: 6.8624421	test: 6.8390981	best: 6.8390981 (3)	total: 1.66s	remaining: 8m 15s
4:	learn: 6.4461365	test: 6.4212617	best: 6.4212617 (4)	total: 2.06s	remaining: 8m 12s
5:	learn: 6.0637835	test: 6.0429915	best: 6.0429915 (5)	total: 2.42s	remaining: 8m
6:	learn: 5.7173595	test: 5.6960670	best: 5.6960670 (6)	total: 2.81s	remaining: 7m 58s
7:	learn: 5.4146341	test: 5.3944491	best: 5.3944491 (7)	total: 3.2s	remaining: 7m 56s
8:	learn: 5.1398552	test: 5.1200175	best: 5.1200175 (8)	total: 3.6s	remaining: 7m 56s
9:	learn: 4.8791347	test: 4.8613489	best: 4.8613489 (9)	total: 3.97s	remaining: 7m 52s
10:	learn: 4.6486444	test: 4.6315129	best: 4.6315129 (10)	total: 4.36s	remaining: 7m 51s
11:	learn: 4.446

In [14]:
from sklearn.metrics import mean_squared_error as mse

MSE = mse(model.predict(X_test), y_test)
print('MSE :', MSE)

MSE : 1.6070182275063034
